In [104]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import responses
import urllib.request
import queue

In [105]:
hz = 5

In [106]:
vehicle_regulation = pd.read_csv('./vehicle_regulation.csv',index_col = 0)

In [107]:
DTGdata = pd.read_csv('./주행 데이터/편현구/편현구.csv',dtype = 'float')

In [108]:
DTGdata.rename(columns={'Unnamed: 0':'time'},inplace=True)

In [110]:
def chagne_heading(heading):
    heading_buffer = abs(heading)*360
    if heading_buffer > 180:
        heading_buffer = 360 - heading_buffer
    return heading_buffer

In [119]:
class long_term_realtime:
    
    def __init__(self,vehicle):
        self.DTGdata = []
        self.vehicle = vehicle
        
    def get_DTGdata(self,new_DTG):
        self.DTGdata.append(new_DTG)
        
    def chagne_heading(self,heading):
        heading_buffer = abs(heading)*360
        if heading_buffer > 180:
            heading_buffer = 360 - heading_buffer
        return heading_buffer
        
    def check_over_speed(self,DTGdata,last_index):
        if (DTGdata['velocity'][last_index] >= vehicle_regulation[self.vehicle]['accel_lim'] + DTGdata['road_limit'][last_index]):
            print('과속 중입니다. 속도를 줄이세요.')
            
    def check_long_accel(self,DTGdata,last_index):
        if (DTGdata[(DTGdata.index >= last_index-vehicle_regulation[self.vehicle]['long_accel_sec']*hz) & 
            (DTGdata['velocity'] <= vehicle_regulation[self.vehicle]['accel_lim'] + DTGdata['road_limit'][last_index])].empty):
            print('장기과속 중입니다. 속도를 줄이세요.')
                        
    def check_rapid_accel(self,DTGdata,last_index):
        if (((DTGdata['velocity'][last_index]-DTGdata['velocity'][last_index-(1*hz)]) >= vehicle_regulation[self.vehicle]['rapid_accel_acceleration']) &
            (DTGdata['velocity'][last_index] >= vehicle_regulation[self.vehicle]['rapid_accel_threshold'])):
            print('급 가속 하였습니다.')
            
    def check_rapid_start(self,DTGdata,last_index):
        if (((DTGdata['velocity'][last_index]-DTGdata['velocity'][last_index-(1*hz)]) >= vehicle_regulation[self.vehicle]['rapid_start_acceleration']) &
            (DTGdata['velocity'][last_index] <= vehicle_regulation[self.vehicle]['rapid_start_threshold'])):
            print('급 출발 하였습니다.')
        
    def check_rapid_decel(self,DTGdata,last_index):
        if (((DTGdata['velocity'][last_index-1]-DTGdata['velocity'][last_index])*hz >= vehicle_regulation[self.vehicle]['rapid_decel_deceleration']) &
            (DTGdata[ 'velocity'][last_index] >= vehicle_regulation[self.vehicle]['rapid_decel_threshold'])):
            print('급 감속 하였습니다.')

    def check_rapid_stop(self,DTGdata,last_index):
        if (((DTGdata['velocity'][last_index]-DTGdata['velocity'][last_index-1*hz]) >= vehicle_regulation[self.vehicle]['rapid_start_acceleration']) &
            (DTGdata['velocity'][last_index] <= vehicle_regulation[self.vehicle]['rapid_start_threshold'])):
            print('급 정지 하였습니다.')
        
    def check_rapid_lanechange(self,DTGdata,last_index):
        if ((DTGdata['velocity'][i] >= vehicle_regulation[self.vehicle]['rapid_lanechange_threshold']) & 
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-1*hz]) >= vehicle_regulation[self.vehicle]['rapid_lanechange_angle']) & 
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-5*hz])/vehicle_regulation[self.vehicle]['rapid_lanechange_time'] <= vehicle_regulation[self.vehicle]['rapid_lanechange_cangle']) &
            (abs(DTGdata['velocity'][last_index] - DTGdata['velocity'][last_index-1*hz]) <= vehicle_regulation[self.vehicle]['rapid_lanechange_defference'])):
            print('급 차선변경 하였습니다.')
                               
    def check_rapid_overtake(self,DTGdata,last_index):
        if ((DTGdata['velocity'][i] >= vehicle_regulation[self.vehicle]['rapid_overtake_threshold']) & 
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-1*hz]) >= vehicle_regulation[self.vehicle]['rapid_overtake_angle']) & 
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-5*hz])/vehicle_regulation[self.vehicle]['rapid_lanechange_time'] <= vehicle_regulation[self.vehicle]['rapid_overtake_cangle']) &
            (abs(DTGdata['velocity'][i] - DTGdata['velocity'][i-1*hz])>= vehicle_regulation[self.vehicle]['rapid_overtake_acceleration'])):
            print('급 앞지르기 하였습니다.')
        
    def check_rapid_turn(self,DTGdata,last_index):
        if ((DTGdata['velocity'][i] >= vehicle_regulation[self.vehicle]['rapid_turn_threshold']) &
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-vehicle_regulation[self.vehicle]['rapid_turn_sec']*hz]) >= vehicle_regulation[self.vehicle]['rapid_turn_angle_start'])&
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-vehicle_regulation[self.vehicle]['rapid_turn_sec']*hz]) <= vehicle_regulation[self.vehicle]['rapid_turn_angle_end'])):
            print('급 회전 하였습니다.')
        
    def check_rapid_uturn(self,DTGdata,last_index):
        if ((DTGdata['velocity'][i] >= vehicle_regulation[self.vehicle]['rapid_uturn_threshold']) &
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-vehicle_regulation[self.vehicle]['rapid_uturn_sec']*hz]) >= vehicle_regulation[self.vehicle]['rapid_uturn_angle_start'])&
            (self.chagne_heading(DTGdata['heading'][last_index] - DTGdata['heading'][last_index-vehicle_regulation[self.vehicle]['rapid_uturn_sec']*hz]) <= vehicle_regulation[self.vehicle]['rapid_uturn_angle_end'])):
            print('급 U턴 하였습니다.')
        
#     def check_long_term(self,DTGdata,last_index):
        
        
    def check_DTGdata(self,new_DTG):
        self.get_DTGdata(new_DTG)
        DTGdata = pd.DataFrame(self.DTGdata,columns = ['time', 'velocity', 'rpm', 'brake', 'gpsX', 'gpsY', 'heading', 'accelX',
       'accelY', 'road_limit'])
        length = len(DTGdata)
        last_index = length-1
        
        if ((last_index >= vehicle_regulation[self.vehicle]['rapid_uturn_sec']*hz)&
             (last_index <= vehicle_regulation[self.vehicle]['long_accel_sec']*hz)):
            self.check_over_speed(DTGdata,last_index)
            self.check_rapid_accel(DTGdata,last_index)
            self.check_rapid_start(DTGdata,last_index)
            self.check_rapid_decel(DTGdata,last_index)
            self.check_rapid_stop(DTGdata,last_index)
            self.check_rapid_lanechange(DTGdata,last_index)
            self.check_rapid_overtake(DTGdata,last_index)
            self.check_rapid_turn(DTGdata,last_index)
            self.check_rapid_uturn(DTGdata,last_index)

        elif((last_index >= vehicle_regulation[self.vehicle]['long_accel_sec']*hz)&
            (last_index <= vehicle_regulation[self.vehicle]['long_term_time']*hz)):
            self.check_over_speed(DTGdata,last_index)
            self.check_long_accel(DTGdata,last_index)
            self.check_rapid_accel(DTGdata,last_index)
            self.check_rapid_start(DTGdata,last_index)
            self.check_rapid_decel(DTGdata,last_index)
            self.check_rapid_stop(DTGdata,last_index)
            self.check_rapid_lanechange(DTGdata,last_index)
            self.check_rapid_overtake(DTGdata,last_index)
            self.check_rapid_turn(DTGdata,last_index)
            self.check_rapid_uturn(DTGdata,last_index)
            
        if(last_index >= vehicle_regulation[self.vehicle]['long_term_time']*hz):
            self.check_over_speed(DTGdata,last_index)
            self.check_long_accel(DTGdata,last_index)
            self.check_rapid_accel(DTGdata,last_index)
            self.check_rapid_start(DTGdata,last_index)
            self.check_rapid_decel(DTGdata,last_index)
            self.check_rapid_stop(DTGdata,last_index)
            self.check_rapid_lanechange(DTGdata,last_index)
            self.check_rapid_overtake(DTGdata,last_index)
            self.check_rapid_turn(DTGdata,last_index)
            self.check_rapid_uturn(DTGdata,last_index)
            check_long_term(DTGdata,last_index)

In [120]:
taxi_realtime = long_term_realtime('taxi')

In [121]:
for i in range(10000):
    taxi_realtime.check_DTGdata(DTGdata.iloc[i].values)

급 출발 하였습니다.
급 정지 하였습니다.
급 출발 하였습니다.
급 정지 하였습니다.
급 가속 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 U턴 하였습니다.
급 회전 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 앞지르기 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 앞지르기 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 감속 하였습니다.
급 회전 하였습니다.
급 감속 하였습니다.
급 회전 하였습니다.
급 감속 하였습니다.
급 회전 하였습니다.
급 감속 하였습니다.
급 회전 하였습니다.
급 감속 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 가속 하였습니다.
급 앞지르기 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습

급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 U턴 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 가속 하였습니다.
급 가속 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 회전 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.


IndexError: single positional indexer is out-of-bounds